In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import pymol
_stdouterr = sys.stdout, sys.stderr
pymol.finish_launching(['/usr/bin/pymol', '-q'])
sys.stdout, sys.stderr = _stdouterr

# load something into the PyMOL window
from pymol import cmd


Gtk-Message: 13:34:48.682: Failed to load module "pk-gtk-module"
Cannot open file '/home/vit/Projects/cryptic-nn/src/prediction-analysis/data/pymol/icons/icon2.svg', because: No such file or directory
Cannot open file '/home/vit/Projects/cryptic-nn/src/prediction-analysis/data/pymol/icons/icon2.svg', because: No such file or directory


Could not read PyMOL stylesheet.
DEBUG: PYMOL_DATA='./data'
 Detected OpenGL version 4.6. Shaders available.
 Geometry shaders not available
 Detected GLSL version 4.60.
PyMOL>set fetch_path, /home/vit/Projects/ahoj2-extraction/src/analysis/cif_files
 Setting: fetch_path set to /home/vit/Projects/ahoj2-extraction/src/analysis/cif_files.


In [ ]:
import json
import sys, os
import numpy as np

from biotite.structure import get_residues
from biotite.structure.io.pdbx import get_structure
import biotite.structure.io.pdbx as pdbx
import biotite.database.rcsb as rcsb
import biotite

sys.path.append('/home/vit/Projects/deeplife-project/src')
import deeplife_utils


DATASET = 'cryptobench'
CIF_FILES = '/home/vit/Projects/deeplife-project/data/cif_files'
PREDICTIONS_PATH = '/home/vit/Projects/cryptic-nn/data/predictions/ESM2-3B-extended-finetuning'

with open(f'../../datasets/{DATASET}-dataset/folds/test.json', 'r') as json_file:
    dataset = json.load(json_file)

skip = True
for apo_structure, holo_structures in dataset.items():
    
    # finished analysis at: '5wbmB' structure
    if apo_structure != '5wbmB' and skip == True:
        skip = False
        continue
    # print(f'Processing {apo_structure} ...')
    binding_residues = set()
    chain_id = holo_structures[0]['apo_chain']

    # skip multichain structures
    if '-' in chain_id:
        continue

    for holo_structure in holo_structures:

        apo_pocket = holo_structure['apo_pocket_selection']
        
        new_apo_residues = [residue.split(
            '_')[1] for residue in apo_pocket]

        binding_residues.update(new_apo_residues)

    cif_file_path = rcsb.fetch(apo_structure, "cif", target_path=CIF_FILES)

    cif_file = pdbx.CIFFile.read(cif_file_path)

    auth = get_structure(cif_file, model=1)
    auth = auth[
            (auth.chain_id == chain_id) &
            (biotite.structure.filter_peptide_backbone(auth))]
    
    protein_id = f'{apo_structure}{chain_id}'
    # skip if no residues left
    if len(auth) == 0: 
        print(f'No residues left for {protein_id}')
        continue

    # filter to get correct chain; filter only for peptides
    auth_residues_only = get_residues(auth)

    predictions = np.load(f'{PREDICTIONS_PATH}/predictions/{protein_id}.npy') > 0.5
    
    assert len(predictions) == len(auth_residues_only[0]), f"Length of auth residues and predictions do not match for {protein_id}: {len(auth_residues_only[0])} vs {len(predictions)}"
    predicted_binding_residues = auth_residues_only[0][predictions]
    true_positives = binding_residues.intersection([str(i) for i in predicted_binding_residues])
    
    cmd.reinitialize()
    cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
    cmd.fetch(protein_id)
    cmd.zoom(protein_id)
    cmd.color('grey', protein_id)
    cmd.color('red',f'{protein_id} and resi {"+".join([str(i) for i in predicted_binding_residues])}')
    cmd.color('blue', f'{protein_id} and resi {"+".join([str(i) for i in binding_residues])}')
    if len(true_positives) > 0:
        cmd.color('green', f'{protein_id} and resi {"+".join([str(i) for i in true_positives])}')
    cmd.show('surface', protein_id)
    input(">Press Enter for the next round...\n")


 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-

## TODO: try also evaluating performance without non-cryptic binding residues

In [43]:
cmd.reinitialize()
cmd.load(cif_file_path, protein_id)

 ExecutiveLoad-Detail: Detected mmCIF
